# Generative Virtual Screening NIM Blueprint
This example notebook demonstrates how to connect BioNeMo NIMs to carry out a few key steps of a virtual screening workflow. Importantly, these steps are powered by highly performant AI models in each category: AlphaFold2 for folding, MolMIM for molecular generation, and DiffDock for protein-ligand docking.

Below, we illustrate this workflow using an example protein and example molecule of interest, the SARS-CoV-2 main protease and Nirmatrelvir, however, the user is free to define any protein and molecule of their choosing.

All of these capabilities are enabled by NVIDIA NIM and NVIDIA NIM Blueprints. For more details, please visit [NVIDIA NIM Blueprints](https://build.nvidia.com/nim/blueprints).

## BioNeMo Configurations
Before you begin, please set the NGC_CLI_API_KEY environment variable to a personal run key for your NGC Org and Team before running docker compose. Then, you can spin the NIMs up using the following docker command from the same directory as the `docker-compose.yaml`:

`docker compose up`

In [3]:
import requests
import numpy

AF2_HOST = 'http://localhost:8081'
DIFFDOCK_HOST = 'http://localhost:8082'
MOLMIM_HOST = 'http://localhost:8083'

## Check health

In [4]:
# AF2
!curl localhost:8081/v1/health/ready

{"status":"ready"}

In [5]:
# Diffdock
!curl localhost:8082/v1/health/ready

true

In [6]:
# MolMIM
!curl localhost:8083/v1/health/ready

{"status":"ready"}

## Utils function

In [7]:
# utility function to prepare (start clean) the output directory
import os, shutil
def prepare_directory(temp):
    """
    Create a new directory and delete the old one if it exists
    :param temp: str: path to the directory
    """
    if os.path.exists(temp):
        # Remove the directory and all its contents
        shutil.rmtree(temp)
    # Recreate the directory
    os.makedirs(temp)

## Protein Folding with AlphaFold2
Once a protein target of interest has been identified, the first step of this virtual screening demonstration is to generate a plausible structure of that protein. We do so by using AlphaFold2, a generative, transformer-based model that translates protein sequences into protein structures.

To demonstrate this part of our workflow, we begin with an example protein. Here, we choose the SARS-CoV-2 main protease as our starting sequence. We query the AlphaFold2 endpoint with this sequence and the model returns the predicted protein structure.

Please note that this step may take 15~20 minutes to be finished (depending on the GPU type), which is much slower than the other two inference steps that usually requires less than a minute

### Fold

In [8]:
# SARS CoV-2 main protease sequence
protein = "SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQ"

In [9]:
# Query AlphaFold2 with sequence above
af2_response = requests.post(
    f'{AF2_HOST}/protein-structure/alphafold2/predict-structure-from-sequence',
    json={
        'sequence': protein,
        'databases': ['uniref90', 'mgnify', 'small_bfd'],
        'msa_algorithm': 'jackhmmer',
        'e_value': 0.0001,
        'bit_score': -1, # -1 means to fallback to the e-value
        'msa_iterations': 1,
        'relax_prediction': True,
    }).json()

In [12]:
len(af2_response)

5

### Write PDB Files

In [27]:
# Write PDB file 
af_output_dir = 'output/alphafold'
prepare_directory(af_output_dir)

for i, structure in enumerate(af2_response):
    fp = os.path.join(af_output_dir, f"prediction_{i}.pdb")
    with open(fp, "w") as f:
      f.write(structure)

### Visualize

In [6]:
from scripts.load_protein_in_notebook import *

In [8]:
# output dir
af_output_dir = 'output/alphafold'

# select id of predicted result
idx = 0

view = load_protein(
    pdb_file_path = os.path.join(af_output_dir, f"prediction_{idx}.pdb"), 
    width=800, 
    height=500
)
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Molecular Generation with MolMIM
The next step in our workflow is generating molecules with optimized chemical properties starting from a seed molecule of interest. Here, molecular generation is powered by MolMIM, an LLM-inspired model aimed at generating and optimizing molecules according to user-defined objectives. The "MIM" part of MolMIM stands for Mutual Information Machine, which describes the mutual-information-based loss used to preserve chemical similarity in the model's latent space.

Here, we begin with Nirmatrelvir, an active component of the Covid treatment Paxlovid, aimed at targeting the SARS-CoV-2 main protease. By using this molecule as the input to MolMIM, the model will return 5 generated molecules with the highest chemical similarity to MolMIM. The user is able to specify the number of generated molecules to return when querying the MolMIM NIM.

Additionally, the user is able to specify chemical properties to optimize for. In this example, we have chosen to optimize the Quantitative Estimate of Drug-Likeness (QED) score, to produce molecules with favorable pharmacokinetic properties.

Note especially that here we're using the `/generate` endpoint of the MolMIM NIM.  But MolMIM was designed for controlled generation with user-defined oracles.  For this type of application you will want to call the `/decode` endpoint.  See the [documentation](https://docs.nvidia.com/nim/bionemo/molmim/latest/overview.html#decode) and [example notebook](https://github.com/NVIDIA/BioNeMo/blob/main/examples/service/notebooks/cma_custom_oracles.ipynb) for additional information about using user-defined oracles.

### Generate

In [10]:
# Nirmatrelvir
molecule = "CC1(C2C1C(N(C2)C(=O)C(C(C)(C)C)NC(=O)C(F)(F)F)C(=O)NC(CC3CCNC3=O)C#N)C"

In [11]:
%%time
molmim_response = requests.post(
    f'{MOLMIM_HOST}/generate',
    json={
        'smi': molecule,
        'num_molecules': 10,
        'algorithm': 'CMA-ES',
        'property_name': 'QED',
        'min_similarity': 0.7, # Ignored if algorithm is not "CMA-ES".
        'iterations': 10,
    }).json()

CPU times: user 0 ns, sys: 5.73 ms, total: 5.73 ms
Wall time: 22.5 s


In [12]:
molmim_response.keys()

dict_keys(['generated'])

In [13]:
# generated molecules
molecules = molmim_response['generated']
molecules

[{'smiles': 'CC(C)(C)C(=O)N1CC(NC(=O)C2(c3ccccc3F)CCC2)C1',
  'score': 0.9249170846136127},
 {'smiles': 'CC(C)(C)C(=O)N1CC(NC(=O)C2([C@H]3CCCCO3)CCC2)C1',
  'score': 0.8654908914496162},
 {'smiles': 'CCNc1ccc(Cl)cc1C(=O)N(CC1CC1)C1CCC1', 'score': 0.854961435643957},
 {'smiles': 'CC(C)Oc1c(N)cccc1C(=O)N1CC(=O)N(C2CCCCC2)C1',
  'score': 0.8513041861006988},
 {'smiles': 'CC(C)(C)C(=O)N1CCN(C(=O)c2ccnn2CC2CCC2)CC1',
  'score': 0.8506698723029753},
 {'smiles': 'CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2(CCCC2)C1',
  'score': 0.8496444654090579},
 {'smiles': 'CC(C)(C)[C@H](NC(=O)C(F)(F)F)C(=O)N1CC2CCC1CC2',
  'score': 0.8496373095154023},
 {'smiles': 'CC(C)(C)[C@H](NC(=O)C(F)(F)c1c(F)cccc1F)C(F)(F)F',
  'score': 0.8195886713917407},
 {'smiles': 'CC(C)(C)[C@H](NC(=O)C(F)(F)c1c(F)cccc1F)C(F)(F)F',
  'score': 0.8195886713917407},
 {'smiles': 'CC(C)(C)C(NC(=O)C(F)(F)c1c(F)cccc1F)C(C)(C)C',
  'score': 0.8027349739886394}]

### Write SDF files

In [14]:
from scripts.utils import *

In [15]:
molmim_output_dir = 'output/molmim'
prepare_directory(molmim_output_dir)

valid_canonical_smiles = convert_smiles_to_sdf(
    smiles_list=[x['smiles'] for x in molecules], 
    output_dir=molmim_output_dir
)

Some SMILES are duplicates and removed
Converted SMILES to SDF: CC(C)(C)C(=O)N1CC(NC(=O)C2([C@H]3CCCCO3)CCC2)C1
Converted SMILES to SDF: CCNc1ccc(Cl)cc1C(=O)N(CC1CC1)C1CCC1
Converted SMILES to SDF: CC(C)(C)C(=O)N1CCN(C(=O)c2ccnn2CC2CCC2)CC1
Converted SMILES to SDF: CC(C)(C)[C@H](NC(=O)C(F)(F)c1c(F)cccc1F)C(F)(F)F
Converted SMILES to SDF: CC(C)Oc1c(N)cccc1C(=O)N1CC(=O)N(C2CCCCC2)C1
Converted SMILES to SDF: CC(C)(C)C(=O)N1CC(NC(=O)C2(c3ccccc3F)CCC2)C1
Converted SMILES to SDF: CC(C)(C)[C@H](NC(=O)C(F)(F)F)C(=O)N1CC2CCC1CC2
Converted SMILES to SDF: CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2(CCCC2)C1
Converted SMILES to SDF: CC(C)(C)C(NC(=O)C(F)(F)c1c(F)cccc1F)C(C)(C)C


In [16]:
generated_ligands = '\n'.join([v for v in valid_canonical_smiles])
generated_ligands

'CC(C)(C)C(=O)N1CC(NC(=O)C2([C@H]3CCCCO3)CCC2)C1\nCCNc1ccc(Cl)cc1C(=O)N(CC1CC1)C1CCC1\nCC(C)(C)C(=O)N1CCN(C(=O)c2ccnn2CC2CCC2)CC1\nCC(C)(C)[C@H](NC(=O)C(F)(F)c1c(F)cccc1F)C(F)(F)F\nCC(C)Oc1c(N)cccc1C(=O)N1CC(=O)N(C2CCCCC2)C1\nCC(C)(C)C(=O)N1CC(NC(=O)C2(c3ccccc3F)CCC2)C1\nCC(C)(C)[C@H](NC(=O)C(F)(F)F)C(=O)N1CC2CCC1CC2\nCC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2(CCCC2)C1\nCC(C)(C)C(NC(=O)C(F)(F)c1c(F)cccc1F)C(C)(C)C'

## Protein-Ligand Docking with DiffDock

After obtaining the molecules with optimized QED scores, we can predict their binding poses to the receptor target. Here, we apply DiffDock, a state-of-the-art generative model that predicts the 3D structure of a protein-ligand complex, to find out the best (most probable) binding poses. A highlighted feature from DiffDock is that a presumed binding pocket, which usually can be characterized only from experimental 3D structures, is not needed (a.k.a., blind-docking). This feature is very useful for AI folded protein structures, as it is able to locate all regions on the protein surface to be bound by drug molecules, providing ingishts for further downstream investigations.

The optimized DiffDock also provides the batch-docking function, by which we can concatenate multiple molecules into one request of docking, each of them will be also sampled for mulitple poses (i.e., num_poses=10 in this example). In the output, the predicted docking poses for each molecule is sorted by a confidence score that inferenced from a confidence model.

### Run docking

In [17]:
# choose a predicted protein to use

protein_file_path = os.path.join('output/alphafold', 'prediction_0.pdb')

with open(protein_file_path, 'r') as file:
    protein_bytes = file.read()

In [18]:
%%time
diffdock_response = requests.post(
    f'{DIFFDOCK_HOST}/molecular-docking/diffdock/generate',
    json={
        'protein': protein_bytes,
        'ligand': generated_ligands,
        'ligand_file_type': 'txt',
        'num_poses': 5,
        'time_divisions': 20,
        'num_steps': 18,
    }).json()

CPU times: user 6.07 ms, sys: 428 μs, total: 6.5 ms
Wall time: 14.1 s


### Write output.json

In [19]:
import json
# change it to your desired output directory. use the `prepare_directory` function to start clean. 
diffdock_output_dir = "output/diffdock" 
prepare_directory(diffdock_output_dir) 

output_json_file = f"{diffdock_output_dir}/output.json"
with open(output_json_file, "w") as f:
    json.dump(diffdock_response, f)

### Write docked poses

In [20]:
from rdkit import Chem
from rdkit.Chem import AllChem, SDWriter

# select indices where status is succes
import numpy as np
status_array = np.array(diffdock_response['status'])
success_indices = np.where(status_array == 'success')[0]

for i in success_indices:
    # e.g. ligand ID will be ligand_0, ligand_1, etc
    ligand_id = 'ligand_' + str(i)

    # get the ligand poses and confidence scores
    ligand_positions = diffdock_response['ligand_positions'][i]
    confidence_scores = diffdock_response['position_confidence'][i]

    # write to SDF file
    for idx, sdf_str in enumerate(ligand_positions):
        mol = Chem.MolFromMolBlock(sdf_str)
        if mol:
            try: 
                # sanitize the molecule
                Chem.SanitizeMol(mol)
                # set the name, like ligand_0_pose_0, ligand_0_pose_1, etc 
                mol_name = ligand_id+f'_pose_{str(idx)}'
                mol.SetProp("_Name", mol_name)
                # set the confidence score as a property
                mol.SetProp("Confidence", str(np.round(confidence_scores[idx], 4)))
                # output SDF file which has all ligands and all poses
                output_sdf_file = os.path.join(diffdock_output_dir, f'{mol_name}.sdf') # output file path
                # create a writer
                writer = SDWriter(output_sdf_file)
                writer.write(mol)
                writer.close()
            except:
                print(f"Failed to sanitize molecule {ligand_id}_pose_{str(idx)}")
                continue



### Visualize

In [21]:
from scripts.load_docked_poses_in_notebook import *

In [22]:
diffdock_output_dir

'output/diffdock'

In [23]:
view = show_docked_poses(
    protein_path = protein_file_path, # predicted target protein file path
    protein_name = 'predicted_protein', 
    ligands_directory = diffdock_output_dir,
    width = 800, 
    height = 600
)

view.show()


Loaded ligand_0_pose_0 with confidence score: -1.998
Loaded ligand_0_pose_1 with confidence score: -2.305
Loaded ligand_0_pose_2 with confidence score: -2.406
Loaded ligand_0_pose_3 with confidence score: -2.482
Loaded ligand_0_pose_4 with confidence score: -3.13
Loaded ligand_1_pose_0 with confidence score: -0.904
Loaded ligand_1_pose_1 with confidence score: -0.999
Loaded ligand_1_pose_2 with confidence score: -2.381
Loaded ligand_1_pose_3 with confidence score: -2.398
Loaded ligand_1_pose_4 with confidence score: -4.114
Loaded ligand_2_pose_0 with confidence score: -1.017
Loaded ligand_2_pose_1 with confidence score: -1.494
Loaded ligand_2_pose_2 with confidence score: -1.892
Loaded ligand_2_pose_3 with confidence score: -2.832
Loaded ligand_2_pose_4 with confidence score: -4.449
Loaded ligand_3_pose_0 with confidence score: -1.732
Loaded ligand_3_pose_1 with confidence score: -1.875
Loaded ligand_3_pose_2 with confidence score: -1.894
Loaded ligand_3_pose_3 with confidence score: -

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In this workflow, we illustrate the ability of BioNeMo NIMs to work in concert to generate meaningful predictions in a small virtual screening workflow. We hope this underscores to the user how easy the tools are to query and assimilate, and how flexible a workflow of this sort can be.